# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
#Suppress pycache file
import sys
sys.dont_write_bytecode = True
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key
print(geoapify_key)

141b8691f629447f822359b7dafbbb82


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,20.29,42,75,1.54,CA,1691261033
1,1,avarua,-21.2078,-159.7750,22.03,78,100,3.60,CK,1691261024
2,2,uttar char fasson,22.2332,90.7132,28.21,84,100,7.13,BD,1691261274
3,3,adamstown,-25.0660,-130.1015,19.13,63,9,6.32,PN,1691261016
4,4,bilibino,68.0546,166.4372,4.17,96,100,2.66,RU,1691261025


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
coordinates = {
    "latitude": city_data_df["Lat"],
    "longitude": city_data_df["Lng"],
    "City": city_data_df["City"],
    "Humidity": city_data_df["Humidity"]
}
# Create a Pandas DataFrame containing coordinates
coordinates_df = pd.DataFrame(coordinates)
# Display sample data
coordinates_df
# Display the map
# Configure the map plot
humidity_map_plot = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)
# Display the map plot
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
# ideal weather = 21 < weather < 28 and humidity < 50
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 28) & (city_data_df["Humidity"] <= 50)]
ideal_weather_df

#null_df = city_data_df[city_data_df["Max Temp"].isnull()]
#null_df
# Drop any rows with null values
final_ideal_weather_df = ideal_weather_df.dropna(subset=["Max Temp", "Humidity"]).reset_index()
final_ideal_weather_df

# Display sample data
final_ideal_weather_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,43,43,cukurca,37.2481,43.6136,23.94,24,0,1.57,TR,1691261283
1,81,81,la sarre,48.8002,-79.1996,23.59,29,5,3.64,CA,1691261027
2,86,86,martuk,50.7475,56.5056,23.93,46,7,2.83,KZ,1691261291
3,107,107,zhosaly,45.4878,64.0781,27.47,42,9,2.72,KZ,1691261295
4,109,109,la ronge,55.1001,-105.2842,23.14,49,40,7.72,CA,1691261068


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
copy_df = final_ideal_weather_df.copy()
hotel_details = {
    "City": copy_df["City"],
    "Country": copy_df["Country"],
    "Latitude": copy_df["Lat"],
    "Longitude": copy_df["Lng"],
    "Humidity": copy_df["Humidity"]
}
hotel_df = pd.DataFrame(hotel_details)
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()

,City,Country,Latitude,Longitude,Humidity,Hotel Name
0,cukurca,TR,37.2481,43.6136,24,
1,la sarre,CA,48.8002,-79.1996,29,
2,martuk,KZ,50.7475,56.5056,46,
3,zhosaly,KZ,45.4878,64.0781,42,
4,la ronge,CA,55.1001,-105.2842,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
# Set the parameters for the type of search
limit = 20
radius = 10000
#filters = f"circle:{longitude},{latitude},{radius}"
#bias = f"proximity:{longitude},{latitude}"

# set up a parameters dictionary
params = {
    "limit":limit,
    "apiKey":geoapify_key    
}
# Print a message to follow up the hotel search
print("Starting hotel search")
print("---------------------------------------")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = f"accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    #pprint(params)
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    #pprint(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Indicate that Data Loading is complete 
print("---------------------------------------")
print("Data Retrieval Complete      ")
print("---------------------------------------")

# Display sample data
hotel_df.head()

Starting hotel search
---------------------------------------
cukurca - nearest hotel: No hotel found
la sarre - nearest hotel: No hotel found
martuk - nearest hotel: No hotel found
zhosaly - nearest hotel: Gostinitsa Yeseta
la ronge - nearest hotel: La Ronge Hotel and Suites
the pas - nearest hotel: Wescana Inn
sorochinsk - nearest hotel: Пирамида
minot - nearest hotel: Hampton Inn & Suites
kapuskasing - nearest hotel: Travelodge
uige - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: Hotel North Two
lucapa - nearest hotel: No hotel found
hargeysa - nearest hotel: No hotel found
concepcion del oro - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
xudat - nearest hotel: Lazy Seal Holiday Cottages Nabran
kununurra - nearest hotel: Kununurra Country Club Resort
khorixas - nearest hotel: iGowati Country Hotel and Lodge
justo daract - nearest hotel: No hotel found
zhezqazghan - nearest hotel: No hotel found
erenhot - nearest hotel: 

,City,Country,Latitude,Longitude,Humidity,Hotel Name
0,cukurca,TR,37.2481,43.6136,24,No hotel found
1,la sarre,CA,48.8002,-79.1996,29,No hotel found
2,martuk,KZ,50.7475,56.5056,46,No hotel found
3,zhosaly,KZ,45.4878,64.0781,42,Gostinitsa Yeseta
4,la ronge,CA,55.1001,-105.2842,49,La Ronge Hotel and Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_coordinates = {
    "latitude": hotel_df["Latitude"],
    "longitude": hotel_df["Longitude"],
    "City": hotel_df["City"],
    #"Humidity": city_data_df["Humidity"]
    "Hotel Name": hotel_df["Hotel Name"],
    "Country": hotel_df["Country"]
}
# Create a Pandas DataFrame containing coordinates
hotel_coordinates_df = pd.DataFrame(hotel_coordinates)
# Display sample data
hotel_coordinates_df
# Display the map
# Configure the map plot
hotel_map_plot = hotel_coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    #size = "Humidity",
    scale = 0.8,
    color = "City",
    hover_cols = ["City", "Country", "Hotel Name"]
)
# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Country,Hotel Name)